In [ ]:
!pip install tensorflow --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.4/615.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.17.0 requires tensorflow<2.18,>=2.17.0, but you have tensorflow 2.18.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import ResNet50, EfficientNetB0, DenseNet121, Xception, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
# Ensure correct image size format
IMAGE_SIZE = (224, 224)  # Tuple format

# Clear previous TensorFlow session
tf.keras.backend.clear_session()

def load_image_paths_labels(folder_path, label):
    image_paths = [os.path.join(folder_path, file) for file in os.listdir(folder_path)]
    labels = [label] * len(image_paths)
    return image_paths, labels

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Dataset Paths
real_path = 'drive/MyDrive/real_and_fake_face/training_real'
fake_path = 'drive/MyDrive/real_and_fake_face/training_fake'

real_paths, real_labels = load_image_paths_labels(real_path, 1)
fake_paths, fake_labels = load_image_paths_labels(fake_path, 0)

image_paths = real_paths + fake_paths
labels = real_labels + fake_labels

df = pd.DataFrame({'Paths': image_paths, 'Real': labels})

def preprocess_image(path, target_size=IMAGE_SIZE):
    img = load_img(path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dims for batch processing
    return img_array / 255.0  # Normalize

df['image'] = df['Paths'].apply(preprocess_image)

# Convert images from Series to NumPy array
X = np.vstack(df['image'].values)
y = np.array(df['Real'].values)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=123)

Mounted at /content/drive


In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    cm = confusion_matrix(y_test, y_pred)
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-Score": f1_score(y_test, y_pred),
        "Confusion Matrix": cm
    }
    return metrics

def plot_metrics(metrics):
    sns.heatmap(metrics["Confusion Matrix"], annot=True, fmt="d", cmap="Blues", xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()
    for key, value in metrics.items():
        if key != "Confusion Matrix":
            print(f"{key}: {value:.4f}")

In [ ]:
# Function to build and train a model
def build_and_train_model(model_name, base_model_fn, preprocess_fn, epochs=20, batch_size=32):
    print(f"\nTraining {model_name}...\n")

    # Clear previous TensorFlow session
    tf.keras.backend.clear_session()

    # Apply model-specific preprocessing
    X_train_prep = preprocess_fn(X_train)
    X_test_prep = preprocess_fn(X_test)

    # Define model architecture
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=output)

    # Compile model
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # Define callbacks
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
    ]

    # Train model
    start_time = time.time()
    history = model.fit(X_train_prep, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test_prep, y_test), callbacks=callbacks)
    print(f"Training time for {model_name}: {time.time() - start_time:.2f} seconds")

    # Save model
    model.save(f"{model_name}.h5")
    print(f"{model_name} saved successfully.\n")

    # Evaluate model
    metrics = evaluate_model(model, X_test_prep, y_test)
    plot_metrics(metrics)



Training DenseNet121...

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 421s 8s/step - accuracy: 0.4846 - loss: 0.7969 - val_accuracy: 0.5306 - val_loss: 0.6915 - learning_rate: 1.0000e-04
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 403s 8s/step - accuracy: 0.5338 - loss: 0.7268 - val_accuracy: 0.5330 - val_loss: 0.6883 - learning_rate: 1.0000e-04
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 402s 8s/step - accuracy: 0.5377 - loss: 0.7255 - val_accuracy: 0.5379 - val_loss: 0.6857 - learning_rate: 1.0000e-04
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 395s 8s/step - accuracy: 0.5543 - loss: 0.7110 - val_accuracy: 0.5770 - val_loss: 0.6847 - learning_rate: 1.0000e-04
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 394s 8s/step - accuracy: 0.5572 - loss: 0.7155 - val_accuracy: 0.5868 - val_loss: 0.6824 - learning_rate: 1.0000e-04
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 396s 8s/step - accuracy: 0.5749 - loss: 0.7063 - val_accuracy: 0.5770 - val_loss: 0.6804 - learning_rate: 1.00

Training time for DenseNet121: 7530.36 seconds
DenseNet121 saved successfully.

13/13 ━━━━━━━━━━━━━━━━━━━━ 87s 6s/step


Accuracy: 0.6186
Precision: 0.6393
Recall: 0.6452
F1-Score: 0.6422


In [ ]:
build_and_train_model("DenseNet121", DenseNet121, tf.keras.applications.densenet.preprocess_input)